This notebook will detail how I build and train a convolutional neural network that classifies a snippet of audio by the type of bat calls and number of bats echolocating in it. 

In [1]:
from keras import layers
from keras import models
from keras.utils import to_categorical

import numpy as np 
import matplotlib.pyplot as plt
plt.rcParams['agg.path.chunksize'] = 10000

Using TensorFlow backend.


In [2]:
model = models.Sequential()
model.add(layers.Dense(15, input_shape=(513*96,)))
model.add(layers.Dense(10, activation='softmax'))

In [3]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 15)                738735    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                160       
Total params: 738,895
Trainable params: 738,895
Non-trainable params: 0
_________________________________________________________________


In [4]:
training_specgms = np.load('training_data/training_specgram.npy')
training_labels = np.load('training_data/training__labels.npy')

testing_specgms = np.load('training_data/testing_specgram.npy')
test_labels = np.load('training_data/testing__labels.npy')

In [5]:
train_specgms = training_specgms.reshape(training_specgms.shape[0],training_specgms.shape[1],training_specgms.shape[2],1)
test_specgms = testing_specgms.reshape(testing_specgms.shape[0],testing_specgms.shape[1],testing_specgms.shape[2],1)

In [6]:
convert_snippet_type_to_categorical = {'none000' : 0,
                                       'none001' : 1,
                                       'single010':2,
                                       'single011':3,
                                       'single100':4,
                                       'single101':5,
                                       'multi010':6,
                                       'multi011':7,
                                       'multi110':8,
                                       'multi111':9 }

In [7]:
training_numlabels = map(lambda X : convert_snippet_type_to_categorical[X], training_labels)
testing_numlabels = map(lambda X : convert_snippet_type_to_categorical[X], test_labels)

In [8]:
train_labels_onehot = to_categorical(training_numlabels)
test_labels_onehot = to_categorical(testing_numlabels)

In [9]:
# need to shuffle all the rows !! 
def shuffle_all_items(specgms, onehot_labels):
    num_items = onehot_labels.shape[0]
    shuf_indices = np.random.choice(range(num_items), num_items, replace=False)
    
    out_specgms = specgms.copy()
    out_specgms = specgms[shuf_indices]
    out_labels = onehot_labels[shuf_indices,:]
    return(out_specgms, out_labels, shuf_indices)
    

In [10]:
train_specgm_shuf, train_onehot_shuf, si = shuffle_all_items(train_specgms, train_labels_onehot)

In [11]:
train_specgm_shuf.shape

(230, 513, 96, 1)

In [12]:
train_specgm_shuf[0].reshape(513*96)

array([0., 0., 0., ..., 0., 0., 0.])

In [13]:
# reshape all the spectrograms:
reshape_to_1d = lambda X : X.reshape(513*96)

oned_train_specgm_shuf = np.zeros((train_specgm_shuf.shape[0], train_specgm_shuf.shape[1]*train_specgm_shuf.shape[2]))
for i, each_im in  enumerate(train_specgm_shuf):
    oned_train_specgm_shuf[i,:] = reshape_to_1d(each_im)
    

In [14]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(oned_train_specgm_shuf, train_onehot_shuf, epochs=15, batch_size=50)

Epoch 1/15
230/230 [==============================] - 1s 3ms/step - loss: 2.4750 - acc: 0.2000
Epoch 2/15
230/230 [==============================] - 0s 1ms/step - loss: 2.2168 - acc: 0.2522
Epoch 3/15
230/230 [==============================] - 0s 1ms/step - loss: 2.2127 - acc: 0.2522
Epoch 4/15
230/230 [==============================] - 0s 1ms/step - loss: 2.2092 - acc: 0.2261
Epoch 5/15
230/230 [==============================] - 0s 1ms/step - loss: 2.2058 - acc: 0.2522
Epoch 6/15
230/230 [==============================] - 0s 1ms/step - loss: 2.2025 - acc: 0.2522
Epoch 7/15
230/230 [==============================] - 0s 1ms/step - loss: 2.1993 - acc: 0.2522
Epoch 8/15
230/230 [==============================] - 0s 1ms/step - loss: 2.1961 - acc: 0.2565
Epoch 9/15
230/230 [==============================] - 0s 1ms/step - loss: 2.1929 - acc: 0.2522
Epoch 10/15
230/230 [==============================] - 0s 1ms/step - loss: 2.1897 - acc: 0.2522
Epoch 11/15
230/230 [============================